Antes de empezar: un poco de documentación para enterder ligeramente cómo funciona HTML:

https://www.w3schools.com/html/html_intro.asp

# Web Scraping: Selenium

En este notebook veremos cómo automatizar la extracción de datos de una página web simulando la navegación. Para ello utilizaremos la librería Selenium:
https://selenium-python.readthedocs.io/

El software libre Selenium es un **framework** para realizar test automatizados de software a aplicaciones web. En principio, fue desarrollado para poner a prueba páginas y apps web, pero el **WebDriver** de Selenium también puede usarse con Python para realizar scraping. Si bien Selenium en sí no está escrito en Python, con este lenguaje de programación es posible acceder a las funciones del software.

A diferencia de Scrapy y de BeautifulSoup, Selenium no trabaja con el texto fuente en HTML de la web en cuestión, sino que carga la página en un navegador sin interfaz de usuario. El navegador interpreta entonces el código fuente de la página y crea, a partir de él, un **Document Object Model** (modelo de objetos de documento o DOM). Esta interfaz estandarizada permite poner a prueba las interacciones de los usuarios. De esta forma se consigue, por ejemplo, simular clics y rellenar formularios automáticamente. Los cambios en la web que resultan de dichas acciones se reflejan en el DOM. La estructura del proceso de web scraping con Selenium es la siguiente:

URL → Solicitud HTTP → HTML → Selenium → DOM

In [1]:
# Imports
import pandas as pd
from selenium import webdriver

Deberemos descargar un archivo Chrome driver para selenium en https://chromedriver.chromium.org/downloads, es importante que compruebes la versión de Chrome que utilizas y descargaues el driver correspondiente.
Una vez descargado crearemos una variable con la dirección completa del path al chromedriver 

El objeto driver es con el que trabajaremos a partir de ahora

In [2]:
# especificamos el path hasta nuestro driver recién descargado:
chrome_driver_path = "/Users/jonathansuarezcaceres/Downloads/1_DataScience/TheBridge/thebridge/week6_EDA_streamlit/day3_webscraping/chromedriver"

options = webdriver.ChromeOptions()
# headless nos permite no tener que ver la navegación en la ventana de Chrome
# Esto es para que el código se ejecute sin la necesidad de abrir una ventana, cambiar de pestaña, etc... Es decir, lo hace todo por debajo sin yo tener que ver toda la ejecución
# options.add_argument("headless")

In [3]:
# Creamos el driver con el que nos vamos a manejar en la sesión de scrapeo:
driver = webdriver.Chrome(executable_path = chrome_driver_path, options = options)

In [4]:
url = "https://www.filmaffinity.com/es/main.html"

driver.get(url)

In [5]:
# indicamos la URL de la página web a la que queremos acceder:

# el objeto driver nos va a permitir alterar el estado del la página
elements_by_tag = driver.find_elements_by_tag_name("button")
elements_by_class = driver.find_element_by_class_name("css-47sehv")
elements_by_xpath = driver.find_element_by_xpath("/html/body/div[1]/div/div/div/div[2]/div/button[2]")

In [6]:
# Atributos y propiedades del objeto que hemos capturado

#dir(elements_by_xpath)
elements_by_xpath.size

{'height': 40, 'width': 340}

In [7]:
elements_by_xpath.get_attribute("size")

'large'

In [8]:
elements_by_xpath.tag_name

'button'

In [9]:
elements_by_xpath.text

'ACEPTO'

In [10]:
for i in range(0, len(elements_by_tag)):
    print(elements_by_tag[i].text)

socios
MÁS OPCIONES
ACEPTO


In [11]:
elements_by_tag

[<selenium.webdriver.remote.webelement.WebElement (session="2e9786e931e8f85dab386c5bc5ccc868", element="27020247-59a3-404e-8b27-33e9607de5ea")>,
 <selenium.webdriver.remote.webelement.WebElement (session="2e9786e931e8f85dab386c5bc5ccc868", element="b5da719a-a36a-4a5b-878e-8a19f24d4b19")>,
 <selenium.webdriver.remote.webelement.WebElement (session="2e9786e931e8f85dab386c5bc5ccc868", element="d02d7297-6e49-4b95-a9f5-b615d2dbbfc5")>]

In [12]:
elements_by_xpath

<selenium.webdriver.remote.webelement.WebElement (session="2e9786e931e8f85dab386c5bc5ccc868", element="d02d7297-6e49-4b95-a9f5-b615d2dbbfc5")>

¡Vaya! Nos hemos encontrado con un pop-up que nos pide aceptar cookies o algo po el estilo, si nos lo queremos saltar, para poder automatizar el proceso deberemos:

1. Buscamos el botón
2. Hacemos click en el botón

Selenium nos permite buscar elementos por etiqueteas de html: https://selenium-python.readthedocs.io/locating-elements.html

CUIDADO! Los metodos de busqueda de elementos estan "duplicados". Tenemos *find_element_by_tag_name* y *find_elements_by_tag_name* el primero nos devolverá el primer elemento que encuentre (aunque haya más), el segundo nos devolverá una lista con todos los elementos que encuentre (aunque solo sea uno)

In [13]:
# Navegando en filmaffinity, en el pop-up:


Una vez tenemos los elementos podemos hacer varias cosas con ellos

Podemos extraer todos los atributos que tenga

In [14]:

# obtenemos todos sus métodos y atributos:

Podemos evaluar que tipo de elemento es (tag)

Podemos sacar el valor que tiene (el texto)

Incluso podemos guardar una imagen del elemento

In [15]:
# Vemos que es tipo 'WebElement' y en la documentación podremos encontrar sus métodos

In [16]:
# guardamos como 'mi_imagen.png' la imagen asociada al xpath
elements_by_xpath.screenshot("mi_imagen.png")

True

Evaluamos que elementos hemos encontrado por el tag:

In [17]:
for index, element in enumerate(elements_by_tag):
    print("Elemento: ", index)
    print("Texto del elemento ", index, " es ", element.text)
    print("El tag del elemento ", index, " es ", element.tag_name)
    element.screenshot("mi_imagen_" + str(index) + ".png")

Elemento:  0
Texto del elemento  0  es  socios
El tag del elemento  0  es  button
Elemento:  1
Texto del elemento  1  es  MÁS OPCIONES
El tag del elemento  1  es  button
Elemento:  2
Texto del elemento  2  es  ACEPTO
El tag del elemento  2  es  button


Nos quedamos con el botón que nos interesa

In [18]:
# Para clickar sobre cosas, la etiqueta que nos sirve generalmente es el tag

boton_aceptar = elements_by_tag[2]
boton_aceptar.click()

Si el elemento es interactivo podremos hacer más cosas además de las anteriores. Por ejemplo: hacer click

Buscamos una película por título

In [19]:
from selenium.webdriver.common.keys import Keys

buscador = driver.find_element_by_xpath("/html/body/div[2]/div[1]/div/div[2]/form/input[1]")

In [20]:
# una vez escrita la búsqueda deberíamos poder activarla:
buscador.send_keys("Interstellar")

In [21]:
buscador.send_keys(Keys.ENTER)

In [22]:
# volvemos a la página anterior
driver.back()

### Vamos a buscar todas las películas que se estrenan el próximo viernes

1. Cogemos los containers que hay en la zona central

In [23]:
menu_lateral = driver.find_element_by_id("lsmenu")
menu_lateral

<selenium.webdriver.remote.webelement.WebElement (session="2e9786e931e8f85dab386c5bc5ccc868", element="621fa8c9-7a96-409d-941c-2a3008e2f9e4")>

In [24]:
secciones_menu = menu_lateral.find_elements_by_tag_name("a")

2. Vemos con cuál nos tenemos que quedar

In [25]:
for a in secciones_menu:
    if a.text == "Próximos estrenos":
        a.click()
        break

Accedemos al container central, en el que aparecen los estrenos por semana que queremos ver, exactamente igual que hemos hecho antes

In [26]:
cajon_central = driver.find_elements_by_id("main-wrapper-rdcat")

In [27]:
cajon_central

[<selenium.webdriver.remote.webelement.WebElement (session="2e9786e931e8f85dab386c5bc5ccc868", element="eb0a5e61-1745-4662-947c-37d41f1c204d")>,
 <selenium.webdriver.remote.webelement.WebElement (session="2e9786e931e8f85dab386c5bc5ccc868", element="586dd351-3983-407d-923f-71ed7479eec7")>,
 <selenium.webdriver.remote.webelement.WebElement (session="2e9786e931e8f85dab386c5bc5ccc868", element="883aa923-eb99-4118-9e4e-a846dc91f1d2")>,
 <selenium.webdriver.remote.webelement.WebElement (session="2e9786e931e8f85dab386c5bc5ccc868", element="6a23e86d-5bd4-4940-9302-5056d1ce38ba")>,
 <selenium.webdriver.remote.webelement.WebElement (session="2e9786e931e8f85dab386c5bc5ccc868", element="a5bf5929-8943-4b8e-9172-0970307bb493")>,
 <selenium.webdriver.remote.webelement.WebElement (session="2e9786e931e8f85dab386c5bc5ccc868", element="f6c946be-132f-41a9-849d-7be51b80ea0a")>,
 <selenium.webdriver.remote.webelement.WebElement (session="2e9786e931e8f85dab386c5bc5ccc868", element="42aa8dae-dbc4-472c-95e9-16

In [28]:
type(cajon_central)

list

In [29]:
for semana in cajon_central:
    print(semana.find_element_by_tag_name("div").text)
    print(semana.find_element_by_tag_name("div").get_attribute("id"))

21 de mayo de 2021
2021-05-21
28 de mayo de 2021
2021-05-28
3 de junio de 2021
2021-06-03
4 de junio de 2021
2021-06-04
11 de junio de 2021
2021-06-11
16 de junio de 2021
2021-06-16
18 de junio de 2021
2021-06-18
25 de junio de 2021
2021-06-25


Buscamos cómo acceder a las películas

In [30]:
for semana in cajon_central:
    fecha = semana.find_element_by_tag_name("div").get_attribute("id")
    if fecha == "2021-05-21":
        break       # Al parar de iterar, nos quedamos con la que cumple la condición

In [31]:
caratulas = semana.find_elements_by_class_name("mc-poster")
lista_pelis = []

for peli in caratulas:
    lista_pelis.append(peli.find_element_by_tag_name("a").get_attribute("href"))

lista_pelis

['https://www.filmaffinity.com/es/film637149.html',
 'https://www.filmaffinity.com/es/film478891.html',
 'https://www.filmaffinity.com/es/film713795.html',
 'https://www.filmaffinity.com/es/film112106.html',
 'https://www.filmaffinity.com/es/film329087.html',
 'https://www.filmaffinity.com/es/film950177.html',
 'https://www.filmaffinity.com/es/film388427.html',
 'https://www.filmaffinity.com/es/film132801.html',
 'https://www.filmaffinity.com/es/film759533.html',
 'https://www.filmaffinity.com/es/film407520.html',
 'https://www.filmaffinity.com/es/film452148.html',
 'https://www.filmaffinity.com/es/film158082.html',
 'https://www.filmaffinity.com/es/film108823.html']

In [32]:
len(lista_pelis)

13

Una vez tenemos todas las urls vamos a ver qué hacemos con cada una de ellas

In [33]:
# Accedemos a la página de la primera pelicula
# Como vamos a abrir una URL, volvemos al driver inicial
driver.get(lista_pelis[0])

In [34]:
#driver.back()

Vamos a ver el proceso que deberíamos hacer con cada una de las películas:

1. Sacamos toda la información que nos interesa

In [35]:
# titulo, nota, numero de votos y ficha técnica
titulo = driver.find_element_by_xpath("/html/body/div[4]/table/tbody/tr/td[2]/div[1]/div[4]/h1/span").text
nota = driver.find_element_by_xpath("/html/body/div[4]/table/tbody/tr/td[2]/div[1]/div[4]/div/div[2]/div[2]/div[1]/div[2]/div[1]").text
votos = driver.find_element_by_xpath("/html/body/div[4]/table/tbody/tr/td[2]/div[1]/div[4]/div/div[2]/div[2]/div[1]/div[2]/div[2]/span").text
ficha = driver.find_element_by_xpath("/html/body/div[4]/table/tbody/tr/td[2]/div[1]/div[4]/div/div[3]/dl[1]")

In [36]:
ficha

<selenium.webdriver.remote.webelement.WebElement (session="2e9786e931e8f85dab386c5bc5ccc868", element="23a360aa-b11b-4889-805d-686ff70ff1a2")>

2. Creamos una lista a partir de la ficha técnica

In [37]:
# Los nombres estan con tag  = 'dt' y los valores con 'dd'
ficha_claves = []
ficha_valores = []

for name in ficha.find_elements_by_tag_name("dt"):
    ficha_claves.append(name.text)

for value in ficha.find_elements_by_tag_name("dd"):
    ficha_valores.append(value.text)

In [38]:
ficha_claves

['Título original',
 'Año',
 'Duración',
 'País',
 'Dirección',
 'Guion',
 'Música',
 'Fotografía',
 'Reparto',
 'Productora',
 'Género',
 'Grupos',
 'Sinopsis']

3. Creamos un dataframe con la info

In [39]:
columns = ["Título", "Nota", "Votos"]
columns.extend(ficha_claves)
len(columns)

16

In [40]:
values = [titulo, nota, votos]
values.extend(ficha_valores)
len(values)

16

In [41]:
pd.DataFrame([values], columns = columns)

,Título,Nota,Votos,Título original,Año,Duración,País,Dirección,Guion,Música,Fotografía,Reparto,Productora,Género,Grupos,Sinopsis
0,Spiral: Saw,"4,9",40,Spiral: From the Book of Saw,2021,93 min.,Estados Unidos,Darren Lynn Bousman,"Josh Stolberg, Pete Goldfinger. Historia: Pete...",Charlie Clouser,Jordan Oram,"Chris Rock, Samuel L. Jackson, Max Minghella, ...","Serendipity Productions, Twisted Pictures, Bur...",Terror. Thriller | Policíaco. Crimen. Asesinos...,Saw,Trabajando a la sombra de un policía veterano ...


Ahora vamos a crear una función que nos haga todo esto para cada una de las películas:

In [50]:
def sacar_info(driver):
    titulo = driver.find_element_by_xpath("/html/body/div[4]/table/tbody/tr/td[2]/div[1]/div[4]/h1/span").text
    try:
        nota = driver.find_element_by_xpath("/html/body/div[4]/table/tbody/tr/td[2]/div[1]/div[4]/div/div[2]/div[2]/div[1]/div[2]/div[1]").text
        votos = driver.find_element_by_xpath("/html/body/div[4]/table/tbody/tr/td[2]/div[1]/div[4]/div/div[2]/div[2]/div[1]/div[2]/div[2]/span").text
    except:
        nota = None
        votos = None

    ficha = driver.find_element_by_xpath("/html/body/div[4]/table/tbody/tr/td[2]/div[1]/div[4]/div/div[3]/dl[1]")

    return titulo, nota, votos, ficha

def sacar_ficha(ficha):
    ficha_claves = []
    ficha_valores = []

    for name in ficha.find_elements_by_tag_name("dt"):
        ficha_claves.append(name.text)

    for value in ficha.find_elements_by_tag_name("dd"):
        ficha_valores.append(value.text)

    return ficha_claves, ficha_valores

def montar_df(ficha_claves, ficha_valores, titulo, nota, votos):
    columns = ["Título", "Nota", "Votos"]
    columns.extend(ficha_claves)

    values = [titulo, nota, votos]
    values.extend(ficha_valores)

    return pd.DataFrame([values], columns = columns)

def nueva_peli(driver):
    titulo, nota, votos, ficha = sacar_info(driver)
    ficha_claves, ficha_valores = sacar_ficha(ficha)
    df_peli = montar_df(ficha_claves, ficha_valores, titulo, nota, votos)

    return df_peli

## Modo Dios: moviendonos entre ventanas

Vamos a ver cómo nos podemos mover entre ventanas del navegador

Abrir nueva ventana:

In [43]:
driver.execute_script("window.open("");")

Movernos a otra ventana

In [52]:
driver.switch_to_window(driver.window_handles[-1])

ipykernel_launcher:1: DeprecationWarning: use driver.switch_to.window instead


Cerrar ventana

In [51]:
# Cuando cerramos una ventana, el driver se queda en la que hemos cerrado, asi que tenemos que decirle a cual moverse nueva
driver.close()

Una vez cerramos la ventana tenemos que indicarle a qué ventana tiene que ir

In [46]:
driver.switch_to_window(driver.window_handles[0])

ipykernel_launcher:1: DeprecationWarning: use driver.switch_to.window instead


Sabiendo cómo podemos movernos por entre las ventanas y sabiendo cómo extraer de cada página toda la información que necesitamos vamos a crear nuestro dataframe:

In [47]:
driver.back()

# para abrir todos los links en lista_pelis
for link in lista_pelis:
    # Abreme una nueva ventana con el link de la pelicula
    driver.execute_script('window.open("'+link+'");")
    # Muevete a la ventana que acabas de abrir
    driver.switch_to.window(driver.window_handles[-1])
    
    driver.get(link)

In [53]:
# Creamos un dataframe con todas las pelis que se estrenan la próxima semana:

df_peliculas = pd.DataFrame()

for link in lista_pelis:
    driver.execute_script('window.open("'+link+'");')
    driver.switch_to.window(driver.window_handles[-1])
    driver.get(link)
    nueva_pelicula = nueva_peli(driver)
    df_peliculas = df_peliculas.append(nueva_pelicula)

¡Tachán! Ya tenemos un dataframe con todas las películas que se van a estrenar el próximo viernes